In [1]:
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.python.keras.preprocessing import sequence
from tensorboard import summary as summary_lib
from sklearn.model_selection import train_test_split

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.13.1


In [2]:
from pymystem3 import Mystem
from tqdm import tqdm_notebook,tqdm_pandas,tqdm
import re
from sklearn.feature_extraction.text import CountVectorizer

from gensim.models import KeyedVectors

import nltk

import tf_metrics
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import text_to_word_sequence

from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences




In [3]:
tqdm_pandas(tqdm)

In [12]:
with open('pikabu.csv') as f:
    print(f.read(500))
       

context,answer
 Убрал 600 литров мусора в реликтовом лесу. Ты просто большой молодец.,Спасибо.
Убрал 600 литров мусора в реликтовом лесу. Ты просто большой молодец. Спасибо.,Приедь к нам в Мурманск пожалуйста.
Убрал 600 литров мусора в реликтовом лесу. Ты просто большой молодец. Спасибо.,Тебе платят за это?
Ты просто большой молодец. Спасибо. Тебе платят за это?,"За посты на Пикабу? Да, платят. Достаточно при написании поста всего лишь .."
"Спасибо. Тебе платят за это? За посты на Пикабу? Да, пл


In [4]:
df_train = pd.read_csv('pikabu.csv',sep=',')

In [5]:
df_train.head(20)

,context,answer
0,Убрал 600 литров мусора в реликтовом лесу. Ты...,Спасибо.
1,Убрал 600 литров мусора в реликтовом лесу. Ты ...,Приедь к нам в Мурманск пожалуйста.
2,Убрал 600 литров мусора в реликтовом лесу. Ты ...,Тебе платят за это?
3,Ты просто большой молодец. Спасибо. Тебе платя...,"За посты на Пикабу? Да, платят. Достаточно при..."
4,Спасибо. Тебе платят за это? За посты на Пикаб...,Не нажимается сук
5,Ты просто большой молодец. Спасибо. Тебе платя...,Лично Путин
6,Спасибо. Тебе платят за это? Лично Путин,дарт вейдар
7,Убрал 600 литров мусора в реликтовом лесу. Ты...,"ну всё, теперь можно не убирать за собой, Чист..."
8,Убрал 600 литров мусора в реликтовом лесу. Ты ...,Зачем заминусили человека? Очевидная ирония же...
9,Убрал 600 литров мусора в реликтовом лесу.,"Герой, про которого я бы читал комиксы от dc и..."


In [14]:
df_train.count()

context    21486165
answer     21486162
dtype: int64

In [45]:
df_train['context'].apply(len).max()

422

In [10]:
context = df_train['context'].values

In [39]:
context.shape

(21486165,)

In [60]:
context[:20]

array([' Убрал 600 литров мусора в реликтовом лесу. Ты просто большой молодец.',
       'Убрал 600 литров мусора в реликтовом лесу. Ты просто большой молодец. Спасибо.',
       'Убрал 600 литров мусора в реликтовом лесу. Ты просто большой молодец. Спасибо.',
       'Ты просто большой молодец. Спасибо. Тебе платят за это?',
       'Спасибо. Тебе платят за это? За посты на Пикабу? Да, платят. Достаточно при написании поста всего лишь ..',
       'Ты просто большой молодец. Спасибо. Тебе платят за это?',
       'Спасибо. Тебе платят за это? Лично Путин',
       ' Убрал 600 литров мусора в реликтовом лесу. Ты просто большой молодец.',
       'Убрал 600 литров мусора в реликтовом лесу. Ты просто большой молодец. ну всё, теперь можно не убирать за собой, Чистомен спасёт!',
       '  Убрал 600 литров мусора в реликтовом лесу.',
       ' Убрал 600 литров мусора в реликтовом лесу. Герой, про которого я бы читал комиксы от dc или марвел!',
       'Убрал 600 литров мусора в реликтовом лесу. Герой

ValueError: could not broadcast input array from shape (10) into shape (9)

In [75]:
np.fromiter((text_prep_tags(text) for text in context[:20]),dtype = )

TypeError: data type not understood

In [107]:
k = np.array(list(map(text_prep_tags,tqdm_notebook(context[:20]))))


KeyError: 'убирать_VERB'

In [108]:
text_id_vocab

{}

In [109]:
id_text_vocab

{4: 'убирать_VERB'}

In [8]:
from nltk.corpus import stopwords
sw_ru = set(stopwords.words('russian'))

In [110]:
mystem = Mystem()

In [99]:
text_id_vocab == {}

True

TypeError: <lambda>() got an unexpected keyword argument 'voc_set'

In [43]:
def index_tokenizer(tokenizer, corpus):
    #voc_set = set()
    res = np.fromiter(map(tokenizer,tqdm_notebook(corpus[:20]),),dtype=str)
    voc_set = np.unique(res)
    print(voc_set)

In [111]:
%%time
np_context = np.apply_along_axis(text_prep_tags,1,np.reshape(context[:100000],(-1,1)))

CPU times: user 29.5 s, sys: 3.13 s, total: 32.6 s
Wall time: 2min 22s


In [106]:
np_context[11100]

array(['решать_VERB', 'сэкономить_VERB', 'еда_NOUN', 'крупный_ADJ',
       'домашний_ADJ', 'хищник_NOUN', 'раздаваться_VERB', 'крик_NOUN',
       'предсмертный_ADJ', 'зачавкать_VERB', 'страшный_ADJ', 'дед_NOUN',
       'старуха_NOUN', 'подходить_VERB', 'окно_NOUN', 'выключать_VERB',
       'свет_NOUN', 'слышать_VERB', 'песня_NOUN', 'узнавать_VERB',
       'киш_NOUN', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', ''

In [49]:
r = np.array([3,4,3,6])
r.shape

(4,)

In [65]:
y = np.empty([400], dtype="str")
y[:r.shape[0]] = r
y

array(['3', '4', '3', '6', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', ''

In [109]:
text_id_vocab = {}
id_text_vocab = {}
prev_word = None

def text_prep_tags(text):
   
    #print(text)
    text = text[0]
    upos_map = {'A':'ADJ','ADV':'ADV','ADVPRO':'ADV','ANUM':'ADJ','APRO':'DET','COM':'ADJ','CONJ':'SCON','INTJ':'INTJ','NONLEX':'X','NUM':'NUM','PART':'PART','PR':'ADP','S':'NOUN','SPRO':'PRON','UNKN':'X' ,'V':'VERB'}
    text = text.lower()
    result = np.array([])
    
    
    # Убираем лишние символы
    #text = re.sub(r'[;,]',r' ',text).strip()
    text = re.sub(r'[^\w\s\.]',r'',text).strip()   
    #text = [token.text for token in razdel.tokenize(text)]
    # Делаем лемматизацию       
#     text = [lemma for lemma in mystem.lemmatize(text) if not lemma.isspace() and lemma not in sw_ru
#             and lemma.strip() not in ['.','..','...']]
    
#     if id_text_vocab != {}:
#         prev_word = id_text_vocab[max(id_text_vocab.keys())]
#     else:       
#         prev_word = None
        
    for item in mystem.analyze(text):
      #  print(item)
        token = None
        if item.get('analysis'):
            lemma = item['analysis'][0]['lex']
            pos = re.split('[=,]', item['analysis'][0]['gr'])[0]
            #and lemma not in sw_ru
            if not lemma.isspace()  and lemma.strip() not in ['.','..','...'] and lemma not in sw_ru:
     
                token = f'{lemma}_{upos_map[pos]}'
        else:
            lem_text = item["text"]
            if not lem_text.isspace() and lem_text.strip() not in ['.','..','...'] and lem_text not in sw_ru:
            
                token = f'{lem_text}_UNKN'
            
        if token:    
            #result.append(token)
            result = np.append(result,token)
            
#             if prev_word:
#                 text_id_vocab[token] = text_id_vocab[prev_word] + 1
#                 id_text_vocab[text_id_vocab[prev_word] + 1] = token
#             else:
#                 text_vocab[token] = 4
#                 id_text_vocab[4] = token
#             prev_word = token

    
    
    
    # Лемматизатор неправильно разбивает слова с дефисом, поэтому исправляем это
#     if '-' in text:
#         for l in range(len(text)):
#             if text[l] == '-':
#                 text[l] = f'{text[l-1]}-{text[l+1]}'
#                 text[l-1] = text[l+1] = text[l]
    zer = np.empty([400], dtype="<U25")
    zer[:result.shape[0]] = result
    return zer

In [ ]:
word2vec = KeyedVectors.load_word2vec_format('../../model.bin', binary=True)

In [82]:
train_token_text = df_train.loc[:100000,'context'].progress_apply(text_prep_tags)
#test_token_text = df_test['text'].progress_apply(text_prep_tags)

100%|██████████| 100001/100001 [02:23<00:00, 698.76it/s]


In [84]:
train_token_text = df_train['context'].progress_apply(text_prep_tags)

  1%|          | 148710/21486165 [03:47<8:19:13, 712.36it/s] 

KeyboardInterrupt: 

In [ ]:
count_v = CountVectorizer(tokenizer=text_prep_tags)
count_v.fit(tqdm_notebook(pd.concat([df_train['text'],df_test['text']])))

In [80]:
train_token_text

0         [None, None, None, None, None, None, None, Non...
1         [None, None, None, None, None, None, None, Non...
2         [None, None, None, None, None, None, None, Non...
3         [т_NOUN, None, None, None, None, None, None, N...
4         [None, None, None, None, None, None, None, Non...
5         [т_NOUN, None, None, None, None, None, None, N...
6         [None, None, None, None, None, None, None, Non...
7         [None, None, None, None, None, None, None, Non...
8         [None, None, None, None, None, None, None, Non...
9         [None, None, None, None, None, None, None, Non...
10        [None, None, None, None, None, None, None, Non...
11        [None, None, None, None, None, None, None, Non...
12        [г_NOUN, None, None, None, None, None, None, N...
13        [д_NOUN, None, None, None, None, None, None, N...
14        [г_NOUN, None, None, None, None, None, None, N...
15        [None, None, None, None, None, None, None, Non...
16        [г_NOUN, None, None, None, Non